We want a trace generating model which sets position over time of the fly according to the fly's (noisy) estimate of relevant variables depending on the reversal index:

$r_0$ and $c$ are fixed inputs. Let $L_F$ be the coordinate of the last food encountered during the activation period.

Here are the ways a model fly could select $r_1$, the second inter-reversal distance. We start at the moment the fly does the first reversal and starts moving in the new direction.

First we need to decide whether the fly will mark (an estimate of) when/where it arrives back at the last food location ($L_F$). 

1. if yes: either one of two values are accumulated as the fly moves towards $L_F$:

    i. $|r_0-d_{LR}|$ (or $|d_{LR}|$)

    ii. $|\theta|$ (or $|\theta - L_F|$)

    The fly establishes that it has reached $L_F$, that is, marked a set point for $L_{FR}$, its estimate of where $L_F$ is when it re-encounters it, when either

    i. $|r_0-d_{LR}|$ crosses $0$  (or $|d_{LR}|$ crosses $r_0$)

    ii. $|\theta|$ crosses $L_F$ (or $|\theta-L_F|$ crosses $0$)

    Then, having marked/stored $L_{FR}$, it counts the steps $m$ since passing $L_{FR}$.
    When $m>c$, it reverses.


2. if no: either one of two values are accumulated as the fly moves towards $L_F$:

    i. $|r_1-d_{LR}|$ (or $|d_{LR}|$)

    ii. $|\theta|$ (or $|\theta - L_F|$)

    The fly reverses when either 

    i. $|r_1-d_{LR}|$ crosses $0$ (or $|d_{LR}|$ crosses $r_1$)

    ii. $|\theta|$ crosses $L_F$ (or $|\theta - L_F|$ crosses $0$)





We want to implement these stored, accumulating values as having a variance that is proportional to their mean.

A simple proposal for how to implement this would be to have at each time step, or at each gait step, we add to the accumulator the true delta value accumulated plus 0-mean noise whose variance is a constant times the current value of the accumulator.




___

Now let's implement the above.

(Case 1: tracking return to $L_F$).

(phase A).

Call the time between the first reversal and marking the return to $L_F$ phase $A$. 
The fly starts (at the first reversal) by storing $r_0$ /  $|\theta|$ / $|\theta - L_F|$ . For phase A, for each update time or update step, the fly updates its estimate of $|r_0-d_{LR}|$ / $|d_{LR}|$ /  $|\theta|$ / $|\theta - L_F|$, by adding to its estimate from the previous step, the true $\Delta {\theta}$ or $\Delta d_{LR}$, plus a zero-centered noise term whose variance is a constant times the estimate from the previous step.

(phase B). 

When the accumuluating value crosses $0$ (or $r_0$ or $L_F$ or $|\theta-L_F|$), a new step or distance counter $m$ begins. 
Similarly, for phase B, for each update time or update step, the fly updates its estimate of $m$, by adding to its estimate from the previous step, the true $\Delta {\theta}$ or $\Delta d_{LR}$, plus a zero-centered noise term whose variance is a constant times the estimate from the previous step.

When the accumuluating value crosses $c$, the fly reverses.

(Case 2: no tracking of return to $L_F$).

The fly starts (at the first reversal) by storing $r_1$ /  $|\theta|$ / $|\theta - L_F|$ . For each update time or update step, the fly updates its estimate of $|r_1-d_{LR}|$ / $|d_{LR}|$ /  $|\theta|$ / $|\theta - L_F|$, by adding to its estimate from the previous step, the true $\Delta {\theta}$ or $\Delta d_{LR}$, plus a zero-centered noise term whose variance is a constant times the estimate from the previous step.

When the accumuluating value crosses $0$ (or $r_1$ or $L_{F2}$ or $|\theta+L_{F2}|$), the fly reverses.







We should end up with
a distribution of reversal locations generated in case 1
and
a distribution of reversal locations generated in case 2
for any given inputted r_0.
Then we compare this distribution to what our real $r_1s$ look like.

Then we do the $i>1$ reversals too according to the case 2 process, and look at the reversal histogram when we do $n_{i_k}$ reversals using this process, as well as just look at the distribution of k_is that is generated. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox,Label

from dance_sim_tools import trace_objects,utility
from dance_sim_tools.utility import histogram_draw_to_parameters as htp
from dance_sim_tools.utility import draw_reversal_distances_tm
from dance_sim_tools.ipywidget_helpers import slider
import pandas as pd
import dance_sim_tools.trace_objects as trace_objects
import warnings
warnings.filterwarnings('ignore')


In [4]:
 def compare_r1_histograms(num_iters=100,dt=0.25,r_0=40.,food_case=0,velocity=10,kappa=0.1,log_num_traces=3):
    #Case 1
    #phase A
    #food_case: 0: 1F, 1: 2F60, 2: #2F90
    #kappa: constant relating variance of position estimate to previous estimate magnitude
    
    
    #Start with the value r_0 which is a fixed input. Fly stores this or theta or theta - L_F 
    #This is the initial value of E_i: estimate at step i of the value it's updating
    num_traces = int(np.exp(log_num_traces))
    t = np.arange(0,num_iters*dt+dt,dt)
    cs = [40,80,110]
    c = cs[food_case]
    r_1_bins = np.linspace(40,200,50)
    num_iters = int(num_iters)

    #Options for what unit/reference point the fly's accumulator works in.
    #0: start at r_0 and subtract as the fly moves (threshold = 0)
    #1: start at 0 and add as the fly moves (threshold = r_0)
    #2: start at -r_0 and add as the fly moves (threshold = 0)
    #3: start at -IFD/2-r_0 and add as the fly moves (threshold = -IFD/2)

    #for now, just implement this as option 1 (that is, E_i starts at 0)

    E_i_1 = np.zeros(num_traces)

    cross_times = np.full(num_traces,np.nan)
    recross_distances = np.full(num_traces,np.nan)
    cum_deltas = np.zeros(num_traces) #This will be the same for every trace until the true 
    already_crossed_inds = np.zeros(num_traces).astype(bool)
    #motion is set to vary (true delta_i)

    #Loop through time steps starting from first reversal
    for i in range(num_iters):
        #Each loop:
        #set a true #/Delta/theta$ or $/Delta d_{LR}$ (signal acquired about body motion 
        #of previous step)
        #(This can be the same each iteration, but can vary across traces and time steps)
        delta_i = velocity*dt
        cum_deltas+=delta_i
        #Add, to E_{i-1} 
        #the true /Delta/theta or /Delta d_{LR},
        E_i  = E_i_1 +delta_i
        #Add to the above a draw from a r.v. with mean 0 and variance = kE_{i-1} --> this is E_i.
        E_i = E_i + np.random.normal(0,kappa*E_i_1,num_traces)
        #(each iteration) check to see if E_i has crossed (0 or r_0 or L_F or |\theta-L_F|)
        #look at the set of E_is. the ones that have crossed the threshold set to nans so nothing
        #happens to them in the later iterations.
        inds_just_crossed = (E_i>r_0) & np.logical_not(already_crossed_inds)
        E_i[inds_just_crossed] = np.nan
        #those who cross: mark the current time = time of threshold crossing
        cross_times[inds_just_crossed] = dt*i
        #and the location it's at when the threshold is crossed
        recross_distances[inds_just_crossed] = cum_deltas[inds_just_crossed]
    #    #reassign E_i_1 to what E_i currently is
        E_i_1 = E_i
        already_crossed_inds = already_crossed_inds & inds_just_crossed

    #print(recross_distances)


    #We'll use recross_distances in combination with overshoot_distances to compute the resulting r_1 

    #once the for loop is ended
    #start phase B

    #Set E_i initially to zero. Fly at this point needs to have access to a stored c.
    E_i_1 = np.zeros(num_traces)
    second_part_distances = np.full(num_traces,np.nan)
    cum_deltas = np.zeros(num_traces) #This will be the same for every trace until the true 
    already_crossed_inds = np.zeros(num_traces).astype(bool)


     #Loop through time steps starting from second crossing of L_F
    for i in range(num_iters):
        #Each loop:
        #set a true #/Delta/theta$ or $/Delta d_{LR}$ (signal acquired about body motion 
        #of previous step)
        #(This can be the same each iteration)
        delta_i = velocity*dt
        cum_deltas+=delta_i
        #Add, to E_{i-1} 
        #the true /Delta/theta$ or $/Delta d_{LR}$
        E_i  = E_i_1 +delta_i
        #Add to the above a draw from a r.v. with mean 0 and variance = kE_{i-1} --> this is E_i.
        E_i = E_i + np.random.normal(0,kappa*E_i_1,num_traces)
             #(each iteration) check to see if E_i has crossed c
        # look at the set of E_is. the ones that have crossed the threshold set to nans so nothing
        # happens to them in the later iterations.
        inds_just_crossed = (E_i>c) & np.logical_not(already_crossed_inds)        

        E_i[inds_just_crossed] = np.nan
        #those who cross: mark the current time = time of threshold crossing
        cross_times[inds_just_crossed] = dt*i
        #and the location it's at when the threshold is crossed
        second_part_distances[inds_just_crossed] = cum_deltas[inds_just_crossed]
        #reassign E_i_1 to what E_i currently is
        E_i_1 = E_i    
        already_crossed_inds = already_crossed_inds & inds_just_crossed



    #Sum second_part_distances+recross_distances
    r_1s = recross_distances+second_part_distances

    plt.figure(figsize=(8,4))
    plt.subplot(121)
    plt.hist(r_1s,bins=r_1_bins)
    #plt.hist(recross_distances,bins=50)
    plt.title('r1 (Split Segments)')

    #Case 2

    #Start with the value r_0 which is a fixed input. 
    #Fly stores one of r_0+c or theta or theta - L_F 
    #This is the initial value of E_i: estimate at step i of the value it's updating

    #Set E_i initially to theta-L_F=0 . Fly at this point needs to have access to a stored r_0+c.
    E_i_1 = np.zeros(num_traces)
    distances = np.full(num_traces,np.nan)
    cum_deltas = np.zeros(num_traces) #This will be the same for every trace until the true 
    already_crossed_inds = np.zeros(num_traces).astype(bool)

    #Loop through time steps starting from first reversal
    for i in range(num_iters):

        #Each loop: 

        #set a true #/Delta/theta$ or $/Delta d_{LR}$ (signal acquired about body motion 
        #of previous step)
        #(This can be the same each iteration)
        delta_i = velocity*dt
        cum_deltas+=delta_i
        #Add, to E_{i-1} 
        #the true /Delta/theta$ or $/Delta d_{LR}$
        E_i  = E_i_1 +delta_i
        #Add to the above a draw from a r.v. with mean 0 and variance = kE_{i-1} --> this is E_i.
        E_i = E_i + np.random.normal(0,kappa*E_i_1,num_traces)
        #(each iteration) check to see if E_i has crossed (r_0 + c)
        # look at the set of E_is. the ones that have crossed the threshold set to nans so nothing
        # happens to them in the later iterations.
        inds_just_crossed = (E_i>r_0+c) & np.logical_not(already_crossed_inds)        

        E_i[inds_just_crossed] = np.nan
        #those who cross: mark the current time = time of threshold crossing
        cross_times[inds_just_crossed] = dt*i
        #and the location it's at when the threshold is crossed
        distances[inds_just_crossed] = cum_deltas[inds_just_crossed]
        #reassign E_i_1 to what E_i currently is
        E_i_1 = E_i    
        already_crossed_inds = already_crossed_inds & inds_just_crossed


    plt.subplot(122)
    plt.hist(distances,bins=r_1_bins)
    plt.title('r1 (Single Segment)')
    #plt.show()

In [7]:
#start,stop,step,init

sn_iters = slider('num_iters',100,1000,100,200)
sdt=slider('dt',0.01,0.5,0.01,0.25)
sr_0=slider('r_0',20.,80.,5.,40.)
svelocity=slider('velocity',5,15,1,10)
skappa=slider('kappa',0.00,.2,0.01,0.02,readout_format='.2f')
slog_num_traces=slider('log_num_traces',3,9,1,4)
condition_toggle = widgets.RadioButtons(options=[('1F',0), ('2F 60',1),
                                          ('2F 90',2)],disabled=False,description='food_case')                                 

sliders = [sn_iters,sdt,sr_0,condition_toggle,svelocity,skappa,slog_num_traces]

items = [Box([slider]) for slider in sliders]
  

ui = Box(items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

slider_names = [slider.description for slider in sliders]
param_dict =  dict(zip(slider_names,sliders))

out = widgets.interactive_output(compare_r1_histograms, param_dict)

display(ui,out)



Box(children=(Box(children=(FloatSlider(value=200.0, continuous_update=False, description='num_iters', max=100…

Output()